In [1]:
import os
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import matplotlib.pyplot as plt
import scipy

from read_scope_data import read_trc_data
from plot_utils import plot_stft

%matplotlib qt

In [4]:
date = "20241029"
file_number = "00001"

# # read data from xray detector scope
# base_dir = os.path.join(r"D:\x-ray", date)
# for ifn in os.listdir(base_dir):
#     if ifn.endswith(f"{file_number}.trc"):
#         if "C1" in ifn:
#             dipole_data, tarr_x = read_trc_data(os.path.join(base_dir, ifn))
#         if "C3" in ifn:
#             vis_data, tarr_x = read_trc_data(os.path.join(base_dir, ifn))
#         if "C4" in ifn:
#             xray_data, tarr_x = read_trc_data(os.path.join(base_dir, ifn))

# read data from Bdot scope
base_dir = os.path.join(r"E:\Bdot", date)
for ifn in os.listdir(base_dir):
    if ifn.endswith(f"{file_number}.trc"):
        if "C1" in ifn:
            By_P21, tarr_B = read_trc_data(os.path.join(base_dir, ifn))
        if "C2" in ifn:
            Bx_P20, tarr_B = read_trc_data(os.path.join(base_dir, ifn))
        if "C3" in ifn:
            By_P20, tarr_B = read_trc_data(os.path.join(base_dir, ifn))

# # read data from magnetron scope
# base_dir = os.path.join(r"D:\magnetron", date)
# for ifn in os.listdir(base_dir):
#     if ifn.endswith(f"{file_number}.trc"):
#         if "C1" in ifn:
#             V1, tarr = read_trc_data(os.path.join(base_dir, ifn))
#             V_bank = np.average(V1)
#             print(f"Bank voltage = {V_bank}")
#         if "C2" in ifn:
#             I_mag, tarr = read_trc_data(os.path.join(base_dir, ifn))
            
#         if "C3" in ifn:
#             P_ref, tarr = read_trc_data(os.path.join(base_dir, ifn))
#             print(f"Read P_ref from {ifn}")

#         if "C4" in ifn:
#             V_mag, tarr = read_trc_data(os.path.join(base_dir, ifn))
        
#         if 'I_mag' in locals() and 'V_mag' in locals():
#             P_mag = -I_mag*0.25*V_mag # 1V = 0.25A; magnetron is negatively biased


Reading data...
Done
Reading data...
Done
Reading data...
Done
Reading data...
Done
Reading data...
Done
Reading data...
Done


In [5]:
times, frequencies, stft_result = plot_stft(
    time_array=tarr_B,
    signal=By_P21,
    samples_per_fft=500000,
    overlap_fraction=0.01,
    window='hanning',
    freq_min=150, # MHz
    freq_max=1000 # MHz
)

In [8]:
plt.figure()
plt.plot(tarr_B*1000, By_P21)
plt.show()


In [5]:
filepath = r"C:\data\LAPD\Bdot_calibration\BDOT-2C-1T-RB1\Raw Cal Data 3Ghz 10-24\bdot-2c-1-bypy-2g.csv"
data_dict = read_NA_data(filepath)


Dictionary keys: dict_keys(['metadata', 'Freq(Hz)', 'S22', 'S23', 'S24', 'S32', 'S33', 'S34', 'S42', 'S43', 'S44'])
